## JSSP Example

This notebook demonstraits how to use prallel tabu seach to generate a an initial population for a genetic algorithm.

In [1]:
from JSSP.data import Data
import os
"""
initialize static data from csv files
"""
data_directory = '../data/data_set2'
Data.initialize_data_from_csv(data_directory + '/sequenceDependencyMatrix.csv',
                              data_directory + '/machineRunSpeed.csv',
                              data_directory + '/jobTasks.csv')

# uncomment this to print the data that was read in
# Data.print_data()

In [2]:
# global variables
runtimes = {
    'tabu search': 30,
    'genetic algorithm': 30
}

output_dir = './jssp_output'

### Tabu Search

In [3]:
from JSSP.tabu_search import TabuSearchManager

# TS parameters
ts_runtime = runtimes['tabu search']
num_processes = 50
tabu_list_size = 50
neighborhood_size = 300
neighborhood_wait = 0.15
probability_change_machine = 0.8

ts_manager = TabuSearchManager(ts_runtime, 
                               num_processes, 
                               tabu_list_size, 
                               neighborhood_size, 
                               neighborhood_wait, 
                               probability_change_machine)

ts_manager.start(progress_bar=True)

print("Best solution found by Tabu Search:")
ts_manager.best_solution.pprint()

Best solution found by Tabu Search:
makespan = 4090.6746987951806
machine_makespans = [4067.1056910569114, 4076.0, 4068.46052631579, 4090.6746987951806, 4065.6216216216217, 4065.7900000000004, 4058.091836734694, 4063.6423841059614]
operation_list =
[[   36     1     0     6  8454]
 [    2     0     0     0 19368]
 [   44     1     0     1 10819]
 ...
 [   30     2     1     6 15242]
 [   32     5     4     3 16111]
 [   24     6     4     4 14392]]


||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| Time: 0:00:30


### Genetic Algorithm 

In [5]:
from JSSP.genetic_algorithm import search
from JSSP.solution import generate_feasible_solution

# GA parameters
ga_runtime = runtimes['genetic algorithm']
population_size = 100
mutation_probability=0.05

# create initial population using solutions found by tabu search
initial_population = ts_manager.all_solutions

# generate random solutions for remaining population
for _ in range(abs(population_size - len(initial_population))):
    initial_population.append(generate_feasible_solution())
    
result = search(ga_runtime, initial_population, mutation_probability, progress_bar=True)

print("Best solution found by Tabu Search:")
result.pprint()

# result.create_schedule(output_dir)

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||| Time: 0:00:29


Best solution found by Tabu Search:
makespan = 4085.6746987951806
machine_makespans = [4052.1056910569114, 4073.0, 4068.46052631579, 4085.6746987951806, 4060.621621621621, 4064.7900000000004, 4040.091836734694, 4061.7382909119924]
operation_list =
[[   36     1     0     6  8454]
 [    2     0     0     0 19368]
 [   44     1     0     1 10819]
 ...
 [   37     4     2     2  5851]
 [   37     5     3     2 18997]
 [   40     3     3     7  9698]]
